ok, on va faire du crack

In [24]:
import pandas as pd

# Créer un fichier CSV contenant tous les mois multiples de 3 depuis 2000
start_date = "2000-03"
end_date = "2024-12"
date_range = pd.date_range(start=start_date, end=end_date, freq="3MS")
base_df = pd.DataFrame({"Date": date_range.strftime("%Y-%m")})

# Sauvegarder le fichier de base
base_file = "base_dates.csv"
base_df.to_csv(base_file, index=False)

# Noms des fichiers CSV
files = [
    "0_cac40_final.csv",
    "0_bce_final.csv",
    "0_confiance_final.csv",
    "0_ipc_final.csv",
    "0_pib_final.csv",
    "0_chomage_final.csv"
]

# Lire le fichier de base pour initialiser le DataFrame fusionné
merged_df = pd.read_csv(base_file)

# Boucle pour fusionner les fichiers restants
for file in files:
    temp_df = pd.read_csv(file)
    temp_df.rename(columns={temp_df.columns[0]: 'Date'}, inplace=True)
    print(f"Fichier en cours de fusion: {file}, colonnes: {list(temp_df.columns)}")
    # Effectuer la jointure
    merged_df = pd.merge(merged_df, temp_df, on='Date', how='outer', suffixes=('', f'_from_{file.split('_')[1]}'))

# Forcer la conversion des dates pour éviter les erreurs
merged_df['Date'] = pd.to_datetime(merged_df['Date'], errors='coerce', format="%Y-%m")

# Vérifier et retirer les dates non convertibles (NaT)
merged_df = merged_df.dropna(subset=['Date'])

# Trier par dates pour s'assurer que tout est en ordre chronologique
merged_df.sort_values(by='Date', inplace=True)

# Combiner les colonnes en double si elles existent
for col in merged_df.columns:
    if '_from_' in col:
        base_col = col.split('_from_')[0]
        if base_col in merged_df.columns:
            merged_df[base_col] = merged_df[base_col].combine_first(merged_df[col])
        merged_df.drop(columns=[col], inplace=True)

# Sauvegarder le résultat dans un fichier CSV final
merged_df.to_csv("merged_final.csv", index=False)

print("Fichiers fusionnés avec succès dans 'merged_final.csv'.")

# Vérification finale des colonnes fusionnées
print("Colonnes finales après fusion:", list(merged_df.columns))


Fichier en cours de fusion: 0_cac40_final.csv, colonnes: ['Date', 'cac40']
Fichier en cours de fusion: 0_bce_final.csv, colonnes: ['Date', 'bce']
Fichier en cours de fusion: 0_confiance_final.csv, colonnes: ['Date', 'confiance']
Fichier en cours de fusion: 0_ipc_final.csv, colonnes: ['Date', 'ipc']
Fichier en cours de fusion: 0_pib_final.csv, colonnes: ['Date', 'pib']
Fichier en cours de fusion: 0_chomage_final.csv, colonnes: ['Date', 'chomage ']
Fichiers fusionnés avec succès dans 'merged_final.csv'.
Colonnes finales après fusion: ['Date', 'cac40', 'bce', 'confiance', 'ipc', 'pib', 'chomage ']


In [29]:
import pandas as pd
import statsmodels.api as sm

# Charger les données à partir du fichier CSV
data = pd.read_csv("merged_final.csv")

# Sélectionner les colonnes numériques pour la régression
# Assurez-vous que 'cac40' est la variable dépendante et les autres sont indépendantes
X = data.drop(columns=['Date', 'cac40',])  # Supprimer les colonnes non numériques ou inutiles
y = data['cac40']

# Ajouter une constante pour le modèle
X = sm.add_constant(X)

# Ajuster le modèle de régression linéaire multiple
model = sm.OLS(y, X).fit()

# Résumé des résultats
print(model.summary())


                            OLS Regression Results                            
Dep. Variable:                  cac40   R-squared:                       0.091
Model:                            OLS   Adj. R-squared:                  0.042
Method:                 Least Squares   F-statistic:                     1.867
Date:                Sun, 29 Dec 2024   Prob (F-statistic):              0.108
Time:                        20:38:07   Log-Likelihood:                -345.07
No. Observations:                  99   AIC:                             702.1
Df Residuals:                      93   BIC:                             717.7
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        -26.6954     28.295     -0.943      0.3

In [30]:
import pandas as pd
import statsmodels.api as sm

# Charger les données à partir du fichier CSV
data = pd.read_csv("merged_final.csv")

# Sélectionner la variable indépendante 'bce' et la variable dépendante 'cac40'
X = data[['pib']]  # Sélectionner uniquement la colonne 'bce'
y = data['cac40']

# Ajouter une constante pour le modèle
X = sm.add_constant(X)

# Ajuster le modèle de régression linéaire simple
model = sm.OLS(y, X).fit()

# Afficher le résumé des résultats
print(model.summary())


                            OLS Regression Results                            
Dep. Variable:                  cac40   R-squared:                       0.031
Model:                            OLS   Adj. R-squared:                  0.021
Method:                 Least Squares   F-statistic:                     3.105
Date:                Sun, 29 Dec 2024   Prob (F-statistic):             0.0812
Time:                        20:38:59   Log-Likelihood:                -348.24
No. Observations:                  99   AIC:                             700.5
Df Residuals:                      97   BIC:                             705.7
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.0709      1.210      1.712      0.0